# Training program for Experiment

## setting path

In [1]:
import sys
import os

sys.path.append(os.path.split(os.getcwd())[0])


## データプレプロセス

## vocablary作成
すでにvocab.txtが作成済みである場合不用

In [2]:
from fast_jtnn.mol_tree import MolTree
from getpass import getpass,getuser

import mysql
from mysql import connector
import warnings

# SQL server profile
host = "localhost"
user = None
passwd = None
port = 3306
database="chemoinfo"

# 
VOCAB_FILE = "./MS_vocab.txt"

# get massbank data from SQL server
try:
    if not isinstance(user,str):
        user = raw_input("user")
    if not isinstance(passwd,str):
        passwd = getpass()
    connect = connector.connect(host=host,user=user,password=passwd,port=port,database=database)
    cursor = connect.cursor()
    cursor.execute("""select smiles from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A'; """)
    smiles_list = cursor.fetchall()
except mysql.connector.Error as e:
    print("Something went wrong: {}".format(e))
    sys.exit(1)
finally:
    if passwd : del passwd
    if connect: connect.close()
    if cursor: cursor.close()

# create vocablary
succes = 0
fault = 0
cset = set()
for one in smiles_list:
    try:
        mol = MolTree(one[0])
    except AttributeError as e:
        warnings.warn("Entered An SMILES that does not meet the rules")
        continue
    for c in mol.nodes:
        cset.add(c.smiles)

# write vocab
with open(VOCAB_FILE,"w") as f:
    for one in cset:
        f.write(one+"\n")

user a
 ·


Something went wrong: 1045 (28000): Access denied for user 'a'@'localhost' (using password: YES)


NameError: name 'connect' is not defined

! mkdir vae_model/
%run ../fast_molvae/vae_train.py --train processed --vocab ./MS_vocab.txt --save_dir vae_model/

# Vocab,datasetのロード

In [2]:
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load
import pickle
import torch

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

'''
MS_Dataset.QUERY = """select smiles,file_path from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A';"""
dataset = MS_Dataset(vocab=vocab,host="localhost",database="chemoinfo",batch_size=20)
'''
train_vali_rate = 0.9

train_dataset, vali_dataset = dataset_load("./massbank.pkl",vocab,20,train_vali_rate)
print("number of train dataset :",len(train_dataset))
print("number of validation dataset :",len(vali_dataset))

('number of train dataset :', 6716)
('number of validation dataset :', 747)


## モデルの作成

In [3]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).to('cuda')
print dec_model
enc_model = ms_peak_encoder_lstm(train_dataset.max_spectrum_size,latent_size).to('cuda')
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
load_model = "./vae_model/model.iter-70000"
dec_model.load_state_dict(torch.load(load_model,map_location='cuda'))
print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(365, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(365, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o): 

## setting optimizer

In [4]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(enc_model.parameters(), lr=1e-3)
#optimizer = optim.SGD(enc_model.parameters(),lr=100)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

In [ ]:
from MS_PredictModel import ms_peak_encoder,MS_Dataset
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

pbar = None
train_dataset.batch_size = 10
vali_dataset.batch_size = 10
def training(max_epoch = 100):
    global pbar
    total_step = 0
    meters = np.zeros(3)
    vali_meters = np.zeros(3)
    with open("log.csv","w") as f:
        f.write("epoch,iter.,word,topo,assm,vali word,vali topo,vali assm\n")
    for epoch in range(max_epoch):
        print("epoch : ",epoch)
        for batch in train_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            total_step+=1
            #pbar.update(1)
            x = x.to('cuda')
            y = y.to('cuda')
            
            enc_model.zero_grad()
            dec_model.zero_grad()
            optimizer.zero_grad()
            
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
            word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
            assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
            total_loss = word_loss+topo_loss+assm_loss
            total_loss.backward()
            optimizer.step()
            
            meters = meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
            if total_step % 200 == 0:
                vali_total = 0
                for batch in vali_dataset:
                    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
                    x = x.to('cuda')
                    y = y.to('cuda')
                    with torch.no_grad():
                        h = enc_model(x,y)
                        tree_vec = h[:,:h.shape[1]/2]
                        mol_vec  = h[:,h.shape[1]/2:]
                        _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
                        word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
                        assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
                        vali_meters = vali_meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
                        vali_total += 1
                meters /= 200
                vali_meters /= vali_total
                print "[%d] , Word: %.2f, Topo: %.2f, Assm: %.2f vali_Word: %.2f, vali_Topo: %.2f, vali_assm: %.2f" % (total_step,meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2])
                with open("log.csv","a") as f:
                    f.write("%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n" % (epoch,total_step,meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2]))
                sys.stdout.flush()
                meters *= 0
                vali_meters *= 0
            if total_step % 200 == 0:
                torch.save(enc_model.state_dict(), "./enc_model" + "/model.iter-" + str(total_step))

#import pdb; pdb.set_trace()
try:
    #if pbar is None:
        #pbar = tqdm()
    training(100)
except RuntimeError as e:
    #if pbar is not None:
        #del pbar
    import traceback
    print(traceback.format_exc())
    #import pdb; pdb.set_trace()
    print(e)



('epoch : ', 0)


/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[200] , Word: 45.32, Topo: 87.56, Assm: 87.94 vali_Word: 47.89, vali_Topo: 88.07, vali_assm: 90.57
[400] , Word: 47.08, Topo: 88.21, Assm: 88.90 vali_Word: 47.54, vali_Topo: 87.93, vali_assm: 90.92
[600] , Word: 47.32, Topo: 87.89, Assm: 88.51 vali_Word: 50.28, vali_Topo: 88.35, vali_assm: 90.64
('epoch : ', 1)
[800] , Word: 48.04, Topo: 88.41, Assm: 89.71 vali_Word: 48.93, vali_Topo: 88.10, vali_assm: 90.86
[1000] , Word: 49.57, Topo: 88.19, Assm: 89.30 vali_Word: 51.69, vali_Topo: 88.59, vali_assm: 91.40
[1200] , Word: 50.25, Topo: 88.32, Assm: 89.26 vali_Word: 50.42, vali_Topo: 88.70, vali_assm: 90.35
('epoch : ', 2)
[1400] , Word: 50.77, Topo: 88.70, Assm: 87.73 vali_Word: 52.56, vali_Topo: 89.06, vali_assm: 90.66
[1600] , Word: 51.22, Topo: 89.32, Assm: 88.62 vali_Word: 51.58, vali_Topo: 89.22, vali_assm: 90.04
[1800] , Word: 51.23, Topo: 89.60, Assm: 88.99 vali_Word: 54.04, vali_Topo: 89.81, vali_assm: 91.21
[2000] , Word: 53.21, Topo: 89.67, Assm: 90.66 vali_Word: 52.72, vali_To

In [42]:
for data in dataset:
    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = data
    data = torch.stack((x,y),2)
    print(data[0])
    indc = data[:,:,1].argsort(1,descending=True)
    print(indc.size())
    data = [torch.index_select(data[batch,:,:],0,indc[batch,:]) for batch in range(data.size()[0])]
    print(data)
    break

tensor([[ 15.0000,   1.0700],
        [ 26.0000,   2.3500],
        [ 27.0000,  27.0700],
        [ 29.0000,  32.9900],
        [ 30.0000,   1.2000],
        [ 31.0000,   2.7400],
        [ 39.0000,  14.0000],
        [ 41.0000,  49.8200],
        [ 42.0000,  11.2000],
        [ 43.0000,  49.3900],
        [ 44.0000,   1.2300],
        [ 45.0000,   2.4800],
        [ 53.0000,   1.2200],
        [ 55.0000,   5.7400],
        [ 56.0000,  11.3800],
        [ 57.0000,  99.9900],
        [ 58.0000,   4.6300],
        [ 59.0000,   4.9900],
        [ 69.0000,   1.2700],
        [ 73.0000,   3.5700],
        [ 74.0000,  25.0000],
        [ 75.0000,   3.5700],
        [ 85.0000,  68.2900],
        [ 86.0000,   4.4400],
        [ 87.0000,  12.8100],
        [102.0000,   3.6500],
        [103.0000,  70.0500],
        [104.0000,   4.1900],
        [115.0000,   1.9600],
        [116.0000,  12.2400],
        [ -1.0000,  -1.0000],
        [ -1.0000,  -1.0000],
        [ -1.0000,  -1.0000],
        [ 